# Ein Überblick

---
Lernziel

- Der Zusammenhang von (logistischer Regression) und Neuronalen Netzwerken
---

Im letzten Notebook werden wir uns noch ein letztes Mal mit verschieden Netzwerk-Architekturen auseinandersetzen und wie diese zusammen hängen.

Dafür werden wir noch einmal mit dem MNIST Datensatz arbeiten. Zunächst laden wir wieder die Daten und normalisieren diese. Wir kodieren die Target-Variable auch wieder zu One-Hot Vektoren.

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
import torch
from torch import nn
from torch import optim
def min_max(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))
def one_hot(x):
    """Die Labels der Bilder müssen noch in Vektoren von Länge 10 codiert werden"""
    dod = len(set(x)) # Checkt wie viele verschieden Ziffern es im Datennsatz gibt
    target = np.zeros([x.shape[0], dod]) # Eine Matrix aus Nullen wird erstellt
    for i in range(x.shape[0]): # Der for-loop setzt eine 1 in die Matrix abhängig davon welches Label das Bild hat
        target[i, x[i]] = 1

    return target

In [ ]:
train_data = np.genfromtxt('../data/mnist/mnist_train.csv', delimiter=',', skip_header =False) #genfromtxt liest .txt Datein, mit delimiter ="," können auch .csv (comma seperated values) Datein einglesen werden  
test_data=np.genfromtxt('../data/mnist/mnist_test.csv', delimiter=',', skip_header =False) # hier lesen wir die Test Daten ein

train_labels=train_data[:,0].astype(int) 
train_images = train_data[:,1:]

test_labels=test_data[:,0].astype(int)
test_images = test_data[:,1:]

In [ ]:
train_targets=one_hot(train_labels)
test_targets = one_hot(test_labels)

In [ ]:
train_images = min_max(train_images)
test_images = min_max(test_images)

In [ ]:
plt.imshow(train_images[0].reshape([28, 28]), cmap="gray")
print("Correct Label: %s" % train_labels[0])

# Lineare Regression

Wir beginnen mit einer simplen Regression. Eine lineare Regression kann man auch als neuronales Netzwerk darstellen.

<img src="Img/summary/lin_reg.png" width ="450px">

Der Output setzt sich aus der gewichteten Summe der Pixelwerte zusammen. Das heißt, jedem Pixel ist ein Gewicht zugeordnet. *Das Neuron kann auch noch ein Bias haben, diese ist aber nicht dargestellt*. 

Da wir nur ein Output Neuron haben, können wir auch nur eine Predicition. Das bedeutet wir können  nur eine binäre Klassifizierung machen. Zum Beispiel: Ist auf dem Bild eine Fünf zu sehen oder nicht?

Wir können  diese lineare Regression auch in Python durchführen.
Dafür benutzen wir die `train_images` als Input und die Spalte der `train_targets`. In dem Fall ist das die fünfte Spalte `train_targets[:,5]`

In [ ]:
linear_reg_model = LinearRegression()
linear_reg_model.fit(train_images, train_targets[:,5])

Wir können uns die Gewichte mit `linear_reg_model.coef_` ausgeben lassen. Insgesamt gibt es 784 Gewichte, für jeden Pixel einen. 

In [ ]:
linear_reg_model.coef_[:5], linear_reg_model.coef_.shape

Um zu schauen, wie gut unsere Model funktioniert, können  wir die `.predict()` Funktion benutzen, um die Wert für unseren Test Datensatz vorherzusagen. Denken Sie daran, dass wir nur Nullen oder Einsen vorhersagen wollen.

`1` = "Fünf"

`0` = "Keine Fünf"

In [ ]:
pred_y = linear_reg_model.predict(test_images)
pred_y

Diese Werte sind weder `0` noch `1`. Wir müssen diese erst noch runden.

In [ ]:
pred_y = np.round(pred_y)
pred_y

Jetzt können wir auch die Accuracy berrechen:

In [ ]:
np.mean(pred_y== test_targets[:,5])

`0.9456` ist gar nicht so schlecht. Denken Sie aber daran, dass nur ungefähr 10% der Bilder eine `5` zeigen. Das heißt, auch das 90% der Bilder keine `5` zeigen. Für diese 90% Prozent müsste unere Model eine `0` vorhersagen, um Korekkt zu sein. Wenn das Modell einfach für alle Bilder eine `0` vorhersagt, würde es auch schon auf eine Accuracy von `0.90` kommen. Dementsprechened ist unsere Accuracy vielleicht weniger spektakulär als angenommen.


Aber ein Problem haben wir noch.Schaeun Sie sich einmal die Vorhersagen für `pred_y[1677]` oder `pred_y[1162]`.

In [ ]:
pred_y[1677],pred_y[1162]

Diese Werte sind weder `1` noch `0`. Wie konnte das passieren?
In einer linearen Regression benutzen wir keine Aktivierungsfunktionen. Deswegen kann der Output einer linearen Regression unbegrenzte Werte annehmen. Wenn die Werte außerhalb von `[-1.5, 1.5]` fallen, werden Sie nicht mehr auf die richtigen Werte gerundet.

Das ist erstmal kein Problem,  wir könnten diesen Werten auch manuell `0`en oder `1`en zuordnen. Aber das Problem, bleibt im Grundsatz bestehen: Wri erlauben es dem Modell Werte Vorherzusagen, die außerhalb des möglichen Bereichs liegen. 

Eine `sigmoid` Funktion verhindert das. Sie transformierte alle Werte in eine Weise, dass Sie dannach zwischen 0 und 1 liegen. Wir können also einfach an die lineare Regression eine `sigmoid` Funktioon "hängen". So würde das Problem gelöst werden. Und genau das passiert bei der logistischen Regression.

<img src="Img/summary/log_reg.png" width="450px">

Auch diese können wir in Python berechnen.

In [ ]:
log_reg_model = LogisticRegression(solver = 'lbfgs', max_iter=1000,  random_state=134)
log_reg_model.fit(train_images, train_targets[:,5])
log_reg_model.coef_[0,256:261], log_reg_model.coef_.shape

Wir erhalten wieder `784` Gewichte. Für jeden Pixel eins. Wir sehene auch, dass unsere Vorhersagen für den Testdatensatz jetzt schon gerundet werden.

In [ ]:
pred_y = log_reg_model.predict(test_images)
pred_y

Auch hier berechnen wir wieder die Accuracy.

In [ ]:
np.mean(pred_y == test_targets[:,5])

Durch das Benutzen einer logistischen Regression konnten wir also die Accuracy steigern. Doch bisher unterscheiden wir nur "Fünf" vs. "keine Fünf". Wir wollen aber eigentlich jede Ziffer erkennen können. Auch das geht mit der logistischen Regression.

Das heißt wir haben mehrere Outputs Nodes. Insgesamt 10 für jede Prediciton eine. 

<img src="Img/summary/log_reg_2.png" width="540px">

Es wird jetzt die `softmax` Funktion  benutzt. Anders als die `sigmoid` Funktion stellt, die `softmax` Funktion sicher, dass die Summe der Activations über die 10 Outputs nicht größere als `1` wird. Würden wir die `sigmoid` Funktion benutzten, könnte es passieren, dass ein Bild als eine fünf und eine eins erkannt wird. 


Für diese logistische Regression müssen wir jetzt die komplette `train_labels` Matrix hinzufügen. 

In [ ]:
log_reg_model_alle = LogisticRegression(solver = 'lbfgs', max_iter=1000,  random_state=134)
log_reg_model_alle.fit(train_images, train_labels)
log_reg_model_alle.coef_[0,256:261], log_reg_model_alle.coef_.shape

Die Weightmatrix `log_reg_model_alle.coef_` hat nun die Größe `[10,784]`. Also pro Outputneuron 784 Weights.

Auch jetzt erhalten wir Vorhersagen, diese enthält die vom Model erkannte Zahl. 

In [ ]:
pred_y = log_reg_model_alle.predict(test_images)
pred_y

Wir berechnen erneut die Accuracy.

In [ ]:
np.mean(pred_y==test_labels)

92,5 % der Ziffer kann dieses Model richtig erkennen. Natürlich schlechter als vorher, aber diesmal ist die Aufgabe viel komplexer, denn es geht nicht nur um eine Ziffer, sondern es gilt alle Zahlen richtig zu erkennen. 
Mit einer einfachen logistischen Regression können wir also eine relativ gute Genauigkeit erreichen. 

Wofür brauchen wir dann noch neuronale Netzwerke? Diese können uns auch die letzten Prozentpunkte in Performance bringen. Der Unterschied zu unserem jetzigen Model und einem neuronalen Netzwerk ist das Fehlen der Hidden Layer. 

<img src="Img/summary/nn1.png" width="450px">

Auch das werden wir mit PyTorch nachbauen:

In [ ]:

train_images =torch.tensor(train_images, dtype = torch.float32)
test_images =torch.tensor(test_images, dtype = torch.float32)

train_labels =torch.tensor(train_labels, dtype = torch.long)
test_labels =torch.tensor(test_labels, dtype = torch.long)

In [ ]:
simple_nn = nn.Sequential(nn.Linear(784,10),nn.ReLU() ,nn.Linear(10,10))
loss_funktion = nn.CrossEntropyLoss()
updater = optim.Adam(simple_nn.parameters(), lr = 0.01)

In [ ]:
torch.manual_seed(1234)
for epoch in range(135):
    updater.zero_grad()
    output = simple_nn(train_images)
    loss = loss_funktion(output, train_labels)
    loss.backward()
    updater.step()
    
    

Der Code sollte Ihnen mittlerweile bekannt sein. Wenn wir uns die Accuracy anschauen sehen wir aber, dass das neuronale Netzwerk eine Accuracy hat, die vergleichbar zur Accuracy der logistische Regression ist.

In [ ]:
pred_y=torch.argmax(simple_nn(test_images),1).detach().numpy()

In [ ]:
np.mean(pred_y==test_labels.numpy())

Das kann mehrere Gründe haben. Grundsätzlich müssen neuronale Netzwerke nicht besser funktionieren als einfachere Modelle.
In dem Fall liegt es aber wahrscheinlich an unserem Modell selber. Wir können mehr oder größere Layers benutzen. Wir können den Optimizer ändern oder auch die Lernrate.


# Übungsaufgabe

Das heute Notebook ist kürzer als sonst, somit haben Sie mehr Zeit für die Übungsaufgabe. 
In der heutigen Übungsaufgabe geht es darum, das gelernte in noch einmal auf den MNIST Datensatz anzuwenden. 

Sie erhalten drei Datensätze (durchmischt):

- Trainingsdaten: benutzen Sie zum Trainieren
- Testdaten: benutzen Sie, um  das trainierte Netzwerk zu evaluieren
- Externer Testdatensatz: Nur Bilder, kein Label → Sie schicken mir die Predictions für diesen Datensatz.

Der Externe Datensatz hat keine Lösungen (zumindest keine, die Sie einsehen können). 
Mit der Übungsaufgabe geben Sie auch **Ihre** Vorhersagen für den Externen Datensatz.

Wir werden dann Ihre Prediction mit den wahren Werten vergleichen. 
*Wer von Ihnen erstellt das beste Modell?*

Einen Anfangsmodell wurde vorgeschrieben.
Sie können von da aus anfangen ihr Netzwerk zu verbessern.

Sie haben verschiedene Möglichkeiten ihr Netzwerk besser zu machen:
Hier  ein paar Beispiele.
- Hyperparameter anpassen z.B. Anzahl Epoch, Batchgröße, Learning Rate oder Anzahl der Hidden Layers
- Batchnorm und Dropout
- CNN
- Optimzer

Passen Sie darauf auf, dass Sie nicht auf den Testdatensatz overfitten. Auch das kann passieren.

Am Ende des Codes ist eine Zelle, mit der Sie die Prediction für den Testdatensatz erstellen und speichern können. 
Dieser wird im Ordner `data` als `meine_prediction.csv` gespeichert.

Bitte reichen Sie sowohl ihre Prediction als auch das Notebook.

# Daten 

Laden Sie zunächst alle Daten.

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
from torch.utils import data
import pandas as pd

def min_max(x):
    return (x - 0.) / (255. - 0.)


In [ ]:
train_data = np.genfromtxt('../data/mnist/mnist_train.csv', delimiter=',', skip_header =False) #genfromtxt liest .txt Datein, mit delimiter ="," können auch .csv (comma seperated values) Datein einglesen werden  

train_labels=train_data[:,0].astype(int) 
train_images = min_max(train_data[:,1:])
del train_data 

test_data=np.genfromtxt('../data/mnist/mnist_test.csv', delimiter=',', skip_header =False) # hier lesen wir die Test Daten ein
test_labels=test_data[:,0].astype(int)
test_images = min_max(test_data[:,1:])

del test_data 
external_images=min_max(np.genfromtxt('../data/mnist/external.csv', delimiter=',', skip_header =False))


In [ ]:
train_images =torch.tensor(train_images, dtype = torch.float32)
test_images =torch.tensor(test_images, dtype = torch.float32)

train_labels =torch.tensor(train_labels, dtype = torch.long)
test_labels =torch.tensor(test_labels, dtype = torch.long)

external_images = torch.tensor(external_images, dtype = torch.float32)

In [ ]:
train_data=data.TensorDataset(train_images, train_labels) # input sind unsere Tensors die einmal die Bilder und einmal die Labels beinhalten
loader = data.DataLoader(train_data, batch_size = 32)

##  Model

In [ ]:
simple_nn = nn.Sequential(nn.Linear(784,10),nn.ReLU() ,nn.Linear(10,10))
loss_funktion = nn.CrossEntropyLoss()
updater = optim.Adam(simple_nn.parameters(), lr = 0.0001)

In [ ]:
torch.manual_seed(1234)
for epoch in range(20):
    simple_nn.train()
    for images, labels in loader:
        updater.zero_grad()
        output = simple_nn(images)
        loss = loss_funktion(output, labels)
        loss.backward()
        updater.step()
    
    simple_nn.eval()
    # EVALUATE #
    # Train
    output = simple_nn(train_images)
    loss = loss_funktion(output, train_labels)
    prediction = torch.argmax(output,1).detach().numpy()
    acc  = np.mean(prediction == train_labels.detach().numpy()  )
    # Tets
    output = simple_nn(test_images)
    test_loss = loss_funktion(output, test_labels)
    prediction = torch.argmax(output,1).detach().numpy()
    test_acc  = np.mean(prediction == test_labels.detach().numpy()  )
    print(f"Epoch {epoch} | Trainings Loss: {loss:.3f} Training Acc: {acc:.3f} | Test Loss: {test_loss:.3f} Test Acc:  {test_acc:.3f}")
        

# Externe Daten 

In [ ]:
simple_nn.eval()
externe_pred = torch.argmax(simple_nn(external_images),1).detach().numpy()

Die nächste Zelle generiert eine `.csv` Datein mit euren Vorhersagen. Diese reicht Ihr bitte mit dem Notebook ein.

In [ ]:
pd.DataFrame(externe_pred.reshape(10000,1)).to_csv("../data/meine_prediction.csv", index =False,header =False)